In [2]:
!pip install torchtext

In [3]:
import bz2
import os.path
import numpy as np
import re

import torchtext
from torchtext.data import get_tokenizer
from torchtext.legacy.data import Field
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torchtext.vocab import FastText, vocab
import torch

import sys
from tqdm import tqdm
from datasets import load_dataset
from datasets import IterableDataset
#from datasets.IterableDataset import shuffle
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM


In [4]:
from datasets import load_dataset
dataset = load_dataset('amazon_polarity')#, split='train')
#shuffled_dataset = dataset.shuffle(seed=42)
#small_dataset = shuffled_dataset.select(range(36000))
#len(small_dataset)
#print(dataset)
#small_dataset



Reusing dataset amazon_polarity (C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4)
100%|██████████| 2/2 [00:01<00:00,  1.87it/s]


In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True)


#tokenized_datasets

In [6]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(50)) 
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(50)) 
big_train_dataset = dataset["train"].shuffle(seed=42).select(range(3000)) 
big_eval_dataset = dataset["test"].shuffle(seed=42).select(range(3000)) 

Loading cached shuffled indices for dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-ea9e58304cf21e85.arrow
Loading cached shuffled indices for dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-951db2bd067ed241.arrow
Loading cached shuffled indices for dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-ea9e58304cf21e85.arrow
Loading cached shuffled indices for dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-951db2bd067ed241.arrow


In [7]:
tokenized_small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)
tokenized_big_train_dataset = big_train_dataset.map(tokenize_function, batched=True)
tokenized_big_eval_dataset = big_eval_dataset.map(tokenize_function, batched=True)


Loading cached processed dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-11bfeacb8a48b394.arrow
100%|██████████| 1/1 [00:00<00:00, 39.61ba/s]
Loading cached processed dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-6aa59c34510d6f8b.arrow
Loading cached processed dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-c630d837a3ff7df2.arrow


In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [9]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments("test_trainer")

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_small_train_dataset, eval_dataset=tokenized_small_eval_dataset
)

In [10]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: content, title.
***** Running training *****
  Num examples = 50
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 21
100%|██████████| 21/21 [15:40<00:00, 36.40s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 21/21 [15:40<00:00, 44.80s/it]

{'train_runtime': 940.8238, 'train_samples_per_second': 0.159, 'train_steps_per_second': 0.022, 'train_loss': 0.5976756867908296, 'epoch': 3.0}


TrainOutput(global_step=21, training_loss=0.5976756867908296, metrics={'train_runtime': 940.8238, 'train_samples_per_second': 0.159, 'train_steps_per_second': 0.022, 'train_loss': 0.5976756867908296, 'epoch': 3.0})

In [11]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
predictions = trainer.predict(tokenized_small_eval_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: content, title.
***** Running Prediction *****
  Num examples = 50
  Batch size = 8
100%|██████████| 7/7 [01:09<00:00,  9.82s/it]

(50, 2) (50,)


In [20]:
preds = np.argmax(predictions.predictions, axis=-1)
metric = load_metric("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

Downloading: 5.78kB [00:00, 1.93MB/s]                   


{'accuracy': 0.44, 'f1': 0.0}

In [22]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    #tokenizer=tokenizer,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: content, title.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 8


ValueError: You have to specify either input_ids or inputs_embeds